# Preparation

In [ ]:
!pip install roboflow
from roboflow import Roboflow
rf = Roboflow(api_key="bgDc5Q3QdkWQmxPqwe40")
project = rf.workspace("parkinglotdetectionteamcentaurus").project("parking-space-4")
version = project.version(1)
dataset = version.download("tfrecord")

In [ ]:
!pip install tensorflow-object-detection-api protobuf==3.20.3

In [ ]:
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import MeanSquaredError
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, Model
from tensorflow.keras.losses import SparseCategoricalCrossentropy, BinaryCrossentropy

# Architecture of the net

In [ ]:
def mbconv(x, expand, out, stride, se_ratio=0.25):
    """
    Mobile Inverted Residual Bottleneck (MBConv) with Squeeze-Excitation.

    Args:
        x: Input tensor
        expand: Expansion factor (1 for no expansion)
        out_channels: Output channels
        stride: Stride (1 or 2)
        se_ratio: Squeeze-Excitation reduction ratio

    Returns:
        Output tensor
    """
    # MBConv block with squeeze-excitation
    in_channels = x.shape[-1]
    identity = x

    # Expansion phase (1x1 Conv)
    if expand > 1:
        x = layers.Conv2D(in_channels * expand, 1, padding='same', use_bias=False)(x)
        x = layers.BatchNormalization()(x)
        x = layers.ReLU(6.0)(x)

    # Depthwise phase (3x3 Conv)
    x = layers.DepthwiseConv2D(3, strides=stride, padding='same', use_bias=False)(x)
    x = layers.BatchNormalization()(x)
    x = layers.ReLU(6.0)(x)

    # Squeeze-Excitation (optional but recommended for parking spot detection)
    if se_ratio:
        # Squeeze
        squeezed = layers.GlobalAveragePooling2D()(x)
        squeezed = layers.Dense(max(1, int(in_channels * se_ratio)),
                              activation='relu')(squeezed)
        # Excite
        squeezed = layers.Dense(in_channels * expand,
                              activation='hard_sigmoid')(squeezed)
        # Reshape and multiply
        x = layers.Multiply()([x, layers.Reshape((1, 1, in_channels * expand))(squeezed)])

    # Projection phase (1x1 Conv)
    x = layers.Conv2D(out, 1, padding='same', use_bias=False)(x)
    x = layers.BatchNormalization()(x)

    # Residual connection if possible
    if stride == 1 and in_channels == out:
        x = layers.Add()([x, identity])

    return x

def build_model(input_shape=(640, 640, 3), num_classes=2):
    inputs = layers.Input(shape=input_shape)

    # Backbone
    x = layers.Conv2D(8, 3, strides=2, padding='same')(inputs)
    x = layers.BatchNormalization()(x)
    x = layers.ReLU(6)(x)

    x = mbconv(x, expand=1, out=8, stride=1)
    tf.print(x.shape)
    x = mbconv(x, expand=6, out=16, stride=2)
    tf.print(x.shape)
    c3 = mbconv(x, expand=6, out=24, stride=2)  # 80x80
    tf.print('c3', c3.shape)
    c4 = mbconv(c3, expand=6, out=32, stride=2) # 40x40
    tf.print('c4', c4.shape)
    c5 = mbconv(c4, expand=6, out=48, stride=2) # 20x20
    tf.print('c5', c5.shape)


    # Detection Heads (Classification + Regression)
    outputs = []
    for i, f in enumerate([c3, c4, c5]):
        # Regression head
        reg = layers.GlobalAveragePooling2D()(f)
        reg = layers.Dense(100 * 4)(reg)
        reg_output = layers.Reshape((100, 4), name=f'reg_{i+1}')(reg)  # <-- NAMED

        # Classification head (NEW)
        cls = layers.GlobalAveragePooling2D()(f)
        cls = layers.Dense(100 * num_classes)(cls)
        cls_output = layers.Reshape((100, num_classes), name=f'cls_{i+1}')(cls)  # <-- NAMED

        outputs.extend([cls_output, reg_output])  # Order matches loss keys

    return Model(inputs=inputs, outputs=outputs)

# Parsing and prepocessing dataset, learning


In [ ]:
def parse_tfrecord(example_proto):
    MAX_OBJECTS = 100  # Set to maximum expected objects per image

    feature_description = {
        'image/encoded': tf.io.FixedLenFeature([], tf.string),
        'image/object/bbox/xmin': tf.io.VarLenFeature(tf.float32),
        'image/object/bbox/xmax': tf.io.VarLenFeature(tf.float32),
        'image/object/bbox/ymin': tf.io.VarLenFeature(tf.float32),
        'image/object/bbox/ymax': tf.io.VarLenFeature(tf.float32),
        'image/object/class/label': tf.io.VarLenFeature(tf.int64),
    }

    example = tf.io.parse_single_example(example_proto, feature_description)

    # Image processing
    image = tf.image.decode_jpeg(example['image/encoded'], channels=3)
    image = tf.image.resize(image, (640, 640))
    image = image / 255.0

    # Bounding box processing
    xmin = tf.sparse.to_dense(example['image/object/bbox/xmin'])
    ymin = tf.sparse.to_dense(example['image/object/bbox/ymin'])
    xmax = tf.sparse.to_dense(example['image/object/bbox/xmax'])
    ymax = tf.sparse.to_dense(example['image/object/bbox/ymax'])
    labels = tf.sparse.to_dense(example['image/object/class/label'])
    labels = labels - 1

    # Convert to center format [x_center, y_center, width, height]
    boxes = tf.stack([
        (xmin + xmax) / 2.0,
        (ymin + ymax) / 2.0,
        xmax - xmin,
        ymax - ymin
    ], axis=-1)

    # [FIX] Pad boxes and labels to fixed size
    boxes = tf.pad(boxes, [[0, MAX_OBJECTS - tf.shape(boxes)[0]], [0, 0]])
    labels = tf.pad(labels, [[0, MAX_OBJECTS - tf.shape(labels)[0]]])

    # [FIX] Set fixed shapes
    boxes.set_shape([MAX_OBJECTS, 4])
    labels.set_shape([MAX_OBJECTS])

    # Format targets for 3 detection heads
    targets = {}
    for i in range(3):  # For 3 detection heads
        targets[f'cls_{i+1}'] = labels
        targets[f'reg_{i+1}'] = boxes

    return image, targets

def create_dataset(tfrecord_path, batch_size=8):
    """Create dataset with padded batching"""
    dataset = tf.data.TFRecordDataset(tfrecord_path)
    dataset = dataset.map(parse_tfrecord)

    # [MODIFIED] Use padded_batch instead of regular batch
    return dataset.repeat().padded_batch(
        batch_size,
        padding_values=(
            tf.constant(0.0, dtype=tf.float32),  # Image padding
            {                                     # Target padding
                'cls_1': tf.constant(-1, dtype=tf.int64),
                'reg_1': tf.constant(0.0, dtype=tf.float32),
                'cls_2': tf.constant(-1, dtype=tf.int64),
                'reg_2': tf.constant(0.0, dtype=tf.float32),
                'cls_3': tf.constant(-1, dtype=tf.int64),
                'reg_3': tf.constant(0.0, dtype=tf.float32)
            }
        )
    ).prefetch(tf.data.AUTOTUNE)
class MaskedSparseCategoricalCrossentropy(tf.keras.losses.Loss):
    def __init__(self, num_classes=2, from_logits=True, name='masked_sparse_cce', **kwargs):
        super().__init__(name=name, **kwargs)
        self.num_classes = num_classes
        self.from_logits = from_logits

    def call(self, y_true, y_pred):

        batch_size = tf.shape(y_pred)[0]
        y_pred = tf.reshape(y_pred, [batch_size, -1, self.num_classes])
        y_true = tf.reshape(y_true, [batch_size, -1])

        mask = tf.not_equal(y_true, -1)
        loss = tf.keras.losses.sparse_categorical_crossentropy(
            y_true, y_pred, from_logits=self.from_logits
        )
        return tf.reduce_mean(tf.boolean_mask(loss, mask))

    def get_config(self):
        config = super().get_config()
        config.update({
            'num_classes': self.num_classes,  # Still include in config
            'from_logits': self.from_logits
        })
        return config

    @classmethod
    def from_config(cls, config):
        # Handle both old and new config versions
        return cls(
            num_classes=config.get('num_classes', 2),  # Safe get with default
            from_logits=config.get('from_logits', True),
            name=config.get('name', 'masked_sparse_cce')
        )

def main():
    # Initialize model
    model = build_model(num_classes=2)

    # Load dataset
    train_dataset = create_dataset("/content/parking-space-4-1/train/vacant_space-NFsp-KKS2-NE6S.tfrecord")
    val_dataset = create_dataset("/content/parking-space-4-1/valid/vacant_space-NFsp-KKS2-NE6S.tfrecord")
    test_dataset = create_dataset("/content/parking-space-4-1/test/vacant_space-NFsp-KKS2-NE6S.tfrecord")

    # Verify dataset
    for images, targets in train_dataset.take(1):
        print("Image shape:", images.shape)  # Should be (8, 640, 640, 3)
        print("Class shapes:", {k: v.shape for k, v in targets.items()})
        # Should all be (8, 100)
        print("Box shapes:", {k: v.shape for k, v in targets.items()})
        # Should all be (8, 100, 4)

    # Define losses with masking
    losses = {
        'cls_1': MaskedSparseCategoricalCrossentropy(num_classes=2),
        'reg_1': MeanSquaredError(),
        'cls_2': MaskedSparseCategoricalCrossentropy(num_classes=2),
        'reg_2': MeanSquaredError(),
        'cls_3': MaskedSparseCategoricalCrossentropy(num_classes=2),
        'reg_3': MeanSquaredError()
    }

    loss_weights = {
        'cls_1': 1.0, 'reg_1': 1.0,
        'cls_2': 1.0, 'reg_2': 1.0,
        'cls_3': 1.0, 'reg_3': 1.0
    }

    callbacks = [
        EarlyStopping(patience=5, monitor='val_loss', restore_best_weights=True),
        ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3),
        ModelCheckpoint('best_model.h5', save_best_only=True)
    ]
    # Compile model
    model.compile(
        optimizer=Adam(learning_rate=1e-4),
        loss=losses,
        loss_weights=loss_weights,
        metrics={'cls_1': 'accuracy', 'cls_2': 'accuracy', 'cls_3': 'accuracy'}
    )
    model.summary()
    # Train

    history = model.fit(
        train_dataset,
        epochs=20,
        steps_per_epoch=100,
        validation_data=val_dataset,
        validation_steps=50,
        callbacks=callbacks
    )

    print("\\nFinal Test Evaluation:")
    model.evaluate(test_dataset, steps=50)

    print("\\nValidation Set Evaluation:")
    model.evaluate(val_dataset, steps=50)

    # Save
    model.save('parking_detector.h5')



if __name__ == "__main__":
    main()
    '''
    # Convert to TFLite (quantization optional)
    converter = tf.lite.TFLiteConverter.from_keras_model(model)
    tflite_model = converter.convert()
    with open('model.tflite', 'wb') as f:
        f.write(tflite_model)
    '''

In [ ]:
def convert_to_tflite(model_path, quantize=True):
    """Converts Keras model to TFLite with float16 quantization"""
    model = tf.keras.models.load_model(
        model_path,
        custom_objects={'MaskedSparseCategoricalCrossentropy': MaskedSparseCategoricalCrossentropy}
    )

    converter = tf.lite.TFLiteConverter.from_keras_model(model)

    if quantize:
        # Float16 quantization (no representative dataset needed)
        converter.optimizations = [tf.lite.Optimize.DEFAULT]
        converter.target_spec.supported_types = [tf.float16]  # Key line
    else:
        # Basic optimization without quantization
        converter.optimizations = [tf.lite.Optimize.OPTIMIZE_FOR_SIZE]

    tflite_model = converter.convert()

    ext = 'fp16.tflite' if quantize else 'basic.tflite'
    with open(f'parking_model_{ext}', 'wb') as f:
        f.write(tflite_model)

# In main() after training:


# Add test inference to verify conversion
def test_tflite_inference():
    interpreter = tf.lite.Interpreter(model_path='parking_model_fp16.tflite')
    interpreter.allocate_tensors()

    # Get input details for camera setup
    input_details = interpreter.get_input_details()
    output_details = interpreter.get_output_details()

    print("\nTFLite Input Requirements:")
    print(f"Input Shape: {input_details[0]['shape']}")
    print(f"Input Type: {input_details[0]['dtype']}")
    print("\nTFLite Output Shapes:")
    for out in output_details:
        print(f"{out['name']}: {out['shape']}")

# Call after conversion

convert_to_tflite('best_model.h5', quantize=True)
test_tflite_inference()

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf

def visualize_predictions(model_path, tfrecord_path, num_samples=4):
    """Visualize dataset samples with ground truth and predictions"""
    # 1. Load model with custom loss handling
    model = tf.keras.models.load_model(
        model_path,
        custom_objects={
            'MaskedSparseCategoricalCrossentropy': MaskedSparseCategoricalCrossentropy
        }
    )

    # 2. Create dataset without batching
    dataset = tf.data.TFRecordDataset(tfrecord_path).map(parse_tfrecord)

    # 3. Process samples
    for image, targets in dataset.take(num_samples):
        # Convert image and add batch dimension
        img_array = tf.expand_dims(image, 0)

        # Get predictions
        predictions = model.predict(img_array)

        # Convert to numpy and denormalize
        img_display = (image.numpy() * 255).astype(np.uint8)

        # Prepare subplots
        fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(20, 10))

        # Plot ground truth
        plot_boxes(ax1, img_display, targets, 'Ground Truth')

        # Plot predictions
        plot_boxes(ax2, img_display, predictions, 'Predictions')

        plt.show()

def plot_boxes(ax, image, data, title):
    """Universal plotting function for both ground truth and predictions"""
    ax.imshow(image)
    ax.set_title(title)
    ax.axis('off')

    # Different handling for ground truth vs predictions
    if isinstance(data, dict):  # Ground truth
        boxes = data['reg_1'].numpy()[0]
        labels = data['cls_1'].numpy()[0]
        confidences = None
    else:  # Predictions
        boxes = data[1][0]  # First regression head
        labels = tf.argmax(data[0][0], axis=-1).numpy()  # First classification head
        confidences = tf.reduce_max(tf.nn.softmax(data[0][0]), axis=-1).numpy()

    # Convert boxes to pixel coordinates
    valid_mask = labels != -1
    boxes = boxes[valid_mask]
    print('boxes: ', boxes)
    labels = labels[valid_mask]
    print('labels: ',labels)
    if confidences is not None:
        confidences = confidences[valid_mask]

    # Convert from center to corner format
    x_center, y_center, width, height = boxes.T * 1280  # Scale to image size
    xmin = x_center - width/2
    ymin = y_center - height/2

    print('xmin :', xmin, '\nymin: ', ymin, '\nwidth: ', '\nheight: ', height)

    # Plot boxes
    for i, (x, y, w, h) in enumerate(zip(xmin, ymin, width, height)):
        color = 'green' if labels[i] == 1 else 'red'  # Assuming 0=free, 1=occupied
        rect = plt.Rectangle((x, y), w, h, fill=False, linewidth=1.5, edgecolor=color)
        ax.add_patch(rect)

        label_text = f'{"Free" if labels[i] == 1 else "Occupied"}'
        if confidences is not None:
            label_text += f' ({confidences[i]:.2f})'
        ax.text(x, y-5, label_text, color=color, fontsize=9,
                bbox=dict(facecolor='white', alpha=0.7, edgecolor='none'))

# Usage
visualize_predictions(
    model_path='/content/best_model.h5',
    tfrecord_path='/content/parking-space-4-1/test/vacant_space-NFsp-KKS2-NE6S.tfrecord',
    num_samples=2
)

# Vallidating and Testing